In [1]:
import os
import pandas as pd
from rdkit import Chem

DATAPATH = "../data"

In [4]:
##compare that the molecule length is the same based on smiles 
import pandas as pd

# Load DrugBank_DDI_interactions.xls file
df = pd.read_excel('../data/DrugBank_DDI_interactions.xls', sheet_name = '003 - final')
df = df[["Smiles", "Outcome"]]

mols = [Chem.MolFromSmiles(s) for s in df["Smiles"].tolist()]
st_smiles = [Chem.MolToSmiles(mol) if mol is not None else None for mol in mols]
inchikey = [Chem.MolToInchiKey(mol) if mol is not None else None for mol in mols]

df["smiles"]= st_smiles
df["inchikey"] = inchikey
print(df.shape)
df.dropna(subset="smiles", inplace=True)
df.rename(columns={"Outcome":"outcome"}, inplace=True)
df = df[["smiles", "inchikey", "outcome"]]
print(df.shape)
df = df.drop_duplicates(keep='first')
print(df.shape)

df.to_csv("../data/training_set.csv", index=False)

(591, 4)
(591, 3)
(589, 3)


# Data Analysis

In [5]:
#TDC dataset
df1 = pd.read_csv('../data/dili_tdc_dataset.csv')
#Our dataset
df2 = pd.read_csv('../data/training_set.csv')

mols = [Chem.MolFromSmiles(s) for s in df1["Drug"].tolist()]
st_smiles = [Chem.MolToSmiles(mol) if mol is not None else None for mol in mols]
inchikey = [Chem.MolToInchiKey(mol) if mol is not None else None for mol in mols]
df1["smiles"]= st_smiles
df1["inchikey"] = inchikey
print(df1.shape)
df1.dropna(subset="smiles", inplace=True)
df1.rename(columns={"Y":"outcome"}, inplace=True)
df1 = df1[["smiles", "inchikey", "outcome"]]
print(df1.shape)

# Merge df1 and df2 on 'InChIKey' column with 'left' join
merged_df = pd.merge(df1, df2[['inchikey']], on='inchikey', how='left', indicator=True)

# Filter out rows present in df2
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column '_merge'
filtered_df = filtered_df.drop(columns='_merge')
# remove duplicates
filtered_df = filtered_df.drop_duplicates(keep='first')
filtered_df.to_csv(os.path.join(DATAPATH, "tdc_test_set.csv"), index=False)

(475, 5)
(475, 3)


In [6]:
overlap_smiles = set(df1['inchikey']).intersection(set(df2['inchikey']))
len(overlap_smiles)

191

In [7]:
# Count the occurrences of positive (1) and negative (0) outcomes in the 'outcome' column
outcome_counts = df2['outcome'].value_counts()

# Print the result
total_samples = len(df2)
print("Number of Negative (0) outcomes:", outcome_counts[0])
print("Number of Positive (1) outcomes:", outcome_counts[1])
print("Total number of samples:", total_samples)

# Calculate and print the ratio of negative (0) and positive (1) outcomes
outcome_ratio = df2['outcome'].value_counts(normalize=True)
print("\nRatio of Negative (0) outcomes:", outcome_ratio[0])
print("Ratio of Positive (1) outcomes:", outcome_ratio[1])


Number of Negative (0) outcomes: 194
Number of Positive (1) outcomes: 395
Total number of samples: 589

Ratio of Negative (0) outcomes: 0.3293718166383701
Ratio of Positive (1) outcomes: 0.6706281833616299


In [2]:
df3 = pd.read_csv('../data/training_set_padel.csv')

In [3]:
df3.head()

,smiles,inchikey,outcome,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
0,OC[C@H]1O[C@H]([S-])[C@H](O)[C@@H](O)[C@@H]1O,JUSMHIGDXPKSID-VFUOTHLCSA-M,1,0,-2.0531,4.215220,33.3588,24.804723,0,0,...,0.590976,0.228891,0.445635,0.341268,0.289659,5.498269,8.553981,18.102394,0.386464,1.076562
1,CN(CCCCCCCCCCN(C)C(=O)Oc1cccc([N+](C)(C)C)c1)C...,RWZVPVOZTJJMNU-UHFFFAOYSA-N,1,0,-4.5182,20.414131,95.8764,98.601236,12,12,...,0.946411,0.031266,0.637683,0.441185,0.404666,90.892754,424.767115,1011.673374,0.919616,1.483534
2,[C-]#N,XFXPMWWXUTWYJX-UHFFFAOYSA-N,1,0,0.1349,0.018198,2.7000,2.860000,0,0,...,0.333333,0.333333,1.000000,1.000000,1.000000,1.687500,0.949219,2.814697,0.000000,3.000000
3,COc1ccc2c(c1)[C@]13CCCC[C@@H]1[C@H](C2)N(C)CC3,MKXZASYAUGDDCJ-NJAFHUGGSA-N,0,0,-0.5910,0.349281,53.9015,50.251825,6,6,...,0.540630,0.340646,0.411617,0.487538,0.463245,11.505295,38.227934,83.032541,0.321914,1.362399
4,CN1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4...,XCGSFFUVFURLIX-VFGNJEKYSA-N,0,0,-2.5214,6.357458,97.3820,90.927755,15,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
